# About the Game

The rules of 3 Snails Isolation are a variation of the original Isolation. In the original form of the game there are two players, each with their own game piece, and a 7-by-7 grid of squares. At the beginning of the game, the first player places their piece on any square. The second player follows suit, and places their piece on any one of the available squares. From that point on, the players alternate turns moving their piece like a queen in chess (any number of open squares vertically, horizontally, or diagonally). When the piece is moved, the square that was previously occupied is blocked, and cannot be used for the remainder of the game. The first player who is unable to move their queen loses.

In this 3 Snails variant, each player controls 3 pieces ("snails"), each of which can only move to 1 surrounding square, either to the left/right or directly above/below it. Once a piece moves away from a square, no other piece can occupy the square again, as in the original Isolation game. For clarity, examine the scenario below:

In the image below, both Q1 and Q2 place their snails on the board. Note, that when choosing your move, you must ensure your moves are selected in proper order


![](./img/sn_1.png)

In the image below, Q1 is in the process of choosing its next set of moves. It has chosen 2 moves already, which are printed in the output box, and still needs to pick where "13" will move. Note that the possible moves for any given piece is either to the left/down or directly above/below it.

![](./img/sn_2.png)

Q1 makes its move and Q2 has already chosen the next set of moves. In the image below, Q2 tried to move two of its snails to the same location. Such moves are illegal, as indicated by the message in the output box.

![](./img/sn_3.png)



You can try playing the game against the Random Player or yourself using the interactive tool below.

# Environment Setup

In [1]:
%run helpers/verify_config.py # verify the environment setup

Your python version is  3.7.11
✅ ALL GOOD


In [5]:
# Following two lines make sure anything imported from .py scripts 
# is automatically reloaded if edited & saved (e.g. local unit tests or players)
%reload_ext autoreload

# %load_ext autoreload
# %autoreload 2
from board_viz import ReplayGame, InteractiveGame
from isolation import Board
from test_players import RandomPlayer, HumanPlayer

import time
from isolation import Board

import player_submission_tests as tests

# Interactive Game & Playback

In [3]:
##### NOTE: Need to click the squares order in which they move

# Remove the first argument ('RandomPlayer()') entirely to play against another HumanPlayer
ig = InteractiveGame(RandomPlayer(), show_legal_moves=True)

GridspecLayout(children=(Button(description=' ', layout=Layout(grid_area='widget001', height='auto', width='au…

Output(layout=Layout(border='1px solid black'))

Game is over, the winner is: Player - Q1


In [4]:
# Here is an example of how to visualise a game replay of 2 random players
game = Board(RandomPlayer(), RandomPlayer(), 7, 7)
winner, move_history, termination = game.play_isolation(time_limit=1000, print_moves=False)

bg = ReplayGame(game, move_history, show_legal_moves=True)
bg.show_board()

GridspecLayout(children=(GridspecLayout(children=(Button(description=' ', layout=Layout(grid_area='widget001',…

# Game AI
---------------------------------

As the game board in Isolation is fully viewable by both players at all times, the game is fully observable, adversarial, and deterministic. This allows the development of an AI-based game playing agent using suitable algorithms and heuristics, including Minimax

## The Minimax Algorithm

Minimax is a decision-making algorithm, **typically used in a turn-based, two player games**. The goal of the algorithm is to find the optimal next move

- **Idea:** Choose move to position with highest minimax value = best achievable payoff against best play
- **Perfect (optimal) play for deterministic, perfect-information games**

### Minimax Algorithm Summary

- In the algorithm, one player is called the maximizer, and the other  player is a minimizer. If we assign an evaluation score to the game board, one player tries to choose a game state with the maximum score, while the other chooses a state with the minimum score


- In other words, the **maximizer works to get the highest score, while the minimizer tries get the lowest score** by trying to counter moves


- It is based on the [zero-sum game](https://en.wikipedia.org/wiki/Zero-sum_game) concept. **In a zero-sum game, the total utility score is divided among the players. An increase in one player's score results into the decrease in another  player's score.** So, the total score is always zero. For one  player to win, the other one has to lose. Examples of such games are chess, poker, checkers, tic-tac-toe


- We traverse the game tree ***depth-first*** until we reach the terminal nodes and assign their parent node a value that is best for the player whose turn it is to move. For example, the game tree of a game of tic-tac-toe looks like:

![](./img/tictactoe-gametree.png)


In [ ]:
## About

In [1]:
#export
class OpenMoveEvalFn:
    def score(self, game, my_player=None, my_turn=None):
        """Score the current game state
        Evaluation function that outputs a score equal to how many
        moves are open for AI player on the board minus how many moves
        are open for Opponent's player on the board.

        Note:
            If you think of better evaluation function, do it in CustomEvalFn below.

            Args
                game (Board): The board and game state.
                my_player (Player object): This specifies which player you are.

            Returns:
                float: The current state's score. MyMoves-OppMoves.

            """
        # Simple crude heuristic which favours the number of possible moves our AI has in comparison
        # to the other player. CustomPlayer is always trying to maximize this value
        # and RandomPlayer is trying to minimimize
        if my_turn:
            my_moves = game.get_active_moves()
            opp_moves = game.get_inactive_moves()

#             num_active_moves_my_player = game.get_player_moves(my_player=my_player)
#             num_active_moves_opponent = game.get_opponent_moves(my_player=my_player)
            
        else:
            my_moves = game.get_inactive_moves()
            opp_moves = game.get_active_moves()
#             num_active_moves_my_player = game.get_opponent_moves(my_player=my_player)
#             num_active_moves_opponent = game.get_player_moves(my_player=my_player)
        
        return  len(my_moves) - len(opp_moves)

######################################################################
########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################
######## IF YOU WANT TO CALL OR TEST IT CREATE A NEW CELL ############
######################################################################
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######
tests.correctOpenEvalFn(OpenMoveEvalFn)
################ END OF LOCAL TEST CODE SECTION ######################

NameError: name 'tests' is not defined

In [8]:
#export
class CustomPlayer:
    # TODO: finish this class!
    """Player that chooses a move using your evaluation function
    and a minimax algorithm with alpha-beta pruning.
    You must finish and test this player to make sure it properly
    uses minimax and alpha-beta to return a good move."""

    def __init__(self, search_depth=3, eval_fn=OpenMoveEvalFn()):
        """Initializes your player.
        
        if you find yourself with a superior eval function, update the default
        value of `eval_fn` to `CustomEvalFn()`
        
        Args:
            search_depth (int): The depth to which your agent will search
            eval_fn (function): Evaluation function used by your agent
        """
        self.eval_fn = eval_fn
        self.search_depth = search_depth
    
    def move(self, game, time_left):
        """Called to determine one move by your agent

        Note:
            1. Do NOT change the name of this 'move' function. We are going to call
            this function directly.
            2. Call alphabeta instead of minimax once implemented.
        Args:
            game (Board): The board and game state.
            time_left (function): Used to determine time left before timeout

        Returns:
            tuple: ((int,int),(int,int),(int,int)): Your best move
        """
        best_move, utility = minimax(self, game, time_left, depth=self.search_depth)
        print(f"AI Player: Moving {best_move} with value {utility}")
        print("---------------------------")
        return best_move

    def utility(self, game, my_turn):
        """You can handle special cases here (e.g. endgame)"""
        return self.eval_fn.score(game, self)



###################################################################
########## DON'T WRITE ANY CODE OUTSIDE THE CLASS! ################
###### IF YOU WANT TO CALL OR TEST IT CREATE A NEW CELL ###########
###################################################################

In [9]:
# dir(Board)

t1_state = [
     [' ', ' ', ' ', ' ', ' ', ' ', ' '],
     [' ', '11', ' ', ' ', ' ', ' ', ' '],
     [' ', ' ', ' ', ' ', '22', ' ', ' '],
     [' ', ' ', '12', ' ', ' ', ' ', ' '],
     [' ', ' ', ' ', ' ', '23', ' ', ' '],
     [' ', '21', ' ', ' ', ' ', '13', ' '],
     [' ', ' ', ' ', ' ', ' ', ' ', ' ']
]

r1 = RandomPlayer(name="R1")
r2 = RandomPlayer(name="R2")
test_game = Board(r1, r2 , 7, 7)
test_game.set_state(t1_state, p1_turn=True)
print(test_game.get_active_player().get_name())
test_game.get_state()

R1


[[' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', '11', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', '22', ' ', ' '],
 [' ', ' ', '12', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', '23', ' ', ' '],
 [' ', '21', ' ', ' ', ' ', '13', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ']]

In [54]:
# Algorithm for finding the best move
def minimax(player, game, time_left, depth, my_turn=True):
    """Implementation of the minimax algorithm.
    Args:
        player (CustomPlayer): This is the instantiation of CustomPlayer()
            that represents your agent. It is used to call anything you
            need from the CustomPlayer class (the utility() method, for example,
            or any class variables that belong to CustomPlayer()).
        game (Board): A board and game state.
        time_left (function): Used to determine time left before timeout
        depth: Used to track how deep you are in the search tree
        my_turn (bool): True if you are computing scores during your turn.

    Returns:
        (tuple, int): best_move, val

    # Note:
        depth --> tells us when to stop searching the tree
    # Steps in the algorithm:
    1. Terminal state check. If at the root node, return the utility of the node/state
    2. Get all possible moves for the current player
    3. Determine if a winner is found for any of the moves from step 2
        a. If the winner is MAX, return the move, utility == +inf
        b. if the winner is MIN, return the move, utility == -inf

    4. If no winner is found, continue searching the game tree by recursively calling MINIMAX to get
    the best forecast value
    5. Set best_move, best_value to be highest utility moves based on forecasted value

    """
#     player.count += 1

    # Determine whose turn it is
    if my_turn:
        ai_player = game.get_active_player()
        cpu_player = game.get_inactive_player()

    else:
        ai_player = game.get_inactive_player()
        cpu_player = game.get_active_player()

    ai_moves = game.get_player_moves(my_player=ai_player)
    cpu_moves = game.get_opponent_moves(my_player=ai_player)

    # Handle time running out
    # TODO
    # print(f"Time left {time_left()}")
    #     if time_left() < 10:
    #         return None, None

    ####################################################################################################
    # Search the game tree
    ####################################################################################################

    # If depth is 0, we know we've arrived at the lowest depth desired. Return number of available moves
    # for the AI - number of moves available for the opponent
    if depth == 0:
        best_value = ai_player.utility(game, my_turn)
        return None, best_value

    if my_turn:
        # Initialize values
        max_value = float("-inf")
        best_move = None

        #         # Get possible moves of the CustomPlayer
        #         ai_moves = game.get_player_moves(my_player=current_player)

        for move in ai_moves:
            #             print(f"Possible Move: {move}")

            # Check all possible moves to see if a winner can be found
            new_board_state, is_over, winner = game.forecast_move(move)

            # Check to see if the game is ended while it is the AI's turn and after the next move
            next_moves_possible = new_board_state.get_player_moves(my_player=ai_player)

            if is_over and len(next_moves_possible) == 0:
                #                 print(f"AI Player: Game is over with move {move}")
                return move, float("inf")

            else:
                #                 print(f"Searching game tree for move {move}")
                # Recursively search through the game tree
                forecasted_move, forecasted_value = minimax(
                    ai_player, new_board_state, time_left, depth=depth - 1, my_turn=False
                )

                if forecasted_value > max_value:
                    #                     print(f"Swapping Max Value {max_value} with forecasted value {forecasted_value} with move {forecasted_move}")
                    max_value = forecasted_value
                    best_move = move
                    # print(f"Found new best move: {best_move} with value {max_value}")

        #         print(f"Returning Best Move {best_move} with value {max_value}")
        #         print()
        return best_move, max_value

    else:  # Opponents turn

        # Initialize values
        min_value = float("inf")
        best_move = None

        #         # Get possible moves of the CustomPlayer
        #         cpu_moves = game.get_opponent_moves(my_player=current_player)

        for move in cpu_moves:
            #             print(f"Possible Move: {move}")

            # Check all possible moves to see if a winner can be found
            new_board_state, is_over, winner = game.forecast_move(move)

            # Check to see if the game is ended while it is the opponents turn and after the next move
            next_moves_possible = new_board_state.get_opponent_moves(my_player=ai_player)

            if is_over and len(next_moves_possible) == 0:
                #                 print(f"Opponent Player: Game is over with move {move}")
                return move, float("-inf")
            else:
                #                 print(f"Searching game tree for move {move}")
                # Recursively search through the game tree
                forecasted_move, forecasted_value = minimax(
                    ai_player, new_board_state, time_left, depth=depth - 1, my_turn=True
                )

                if forecasted_value < min_value:
                    #                     print(f"Swapping Max Value {min_value} with forecasted value {forecasted_value} with move {forecasted_move}")
                    min_value = forecasted_value
                    best_move = move
                    print(f"Finding Opponents best move: {best_move} with value {min_value}")

        #         print(f"Returning Best Move {best_move} with value {min_value}")
        #         print()
        return best_move, min_value
    
        


######################################################################
########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################
######## IF YOU WANT TO CALL OR TEST IT CREATE A NEW CELL ############
######################################################################
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######
tests.beatRandom(CustomPlayer)
tests.algorithmTest(CustomPlayer, minimax, "Minimax")
################ END OF LOCAL TEST CODE SECTION ######################



 RandomPlayer - Q1  Turn
move chosen: Q1 to (3, 6), Q2 to (2, 6), and Q3 to (1, 4)
  |0 |1 |2 |3 |4 |5 |6 |
0 |21|  |  |  |  |  |  |
1 |  |  |  |  |13|  |  |
2 |  |  |  |  |><|><|12|
3 |  |  |23|  |  |><|11|
4 |  |  |  |  |  |22|  |
5 |  |  |  |  |  |  |  |
6 |  |  |  |  |  |  |  |

 CustomPlayer - Q2  Turn
Finding Opponents best move: ((4, 6), (1, 6), (0, 4)) with value 14
Finding Opponents best move: ((4, 6), (1, 6), (1, 3)) with value 12
Finding Opponents best move: ((4, 6), (1, 6), (0, 4)) with value 14
Finding Opponents best move: ((4, 6), (1, 6), (1, 3)) with value 12
Finding Opponents best move: ((4, 6), (1, 6), (0, 4)) with value 14
Finding Opponents best move: ((4, 6), (1, 6), (1, 3)) with value 8
Finding Opponents best move: ((4, 6), (1, 6), (0, 4)) with value 14
Finding Opponents best move: ((4, 6), (1, 6), (1, 3)) with value 8
Finding Opponents best move: ((4, 6), (1, 6), (0, 4)) with value 23
Finding Opponents best move: ((4, 6), (1, 6), (1, 3)) with value 21
Finding Opp

Finding Opponents best move: ((3, 4), (5, 3), (4, 1)) with value 1
Finding Opponents best move: ((3, 4), (5, 3), (4, 1)) with value 3
Finding Opponents best move: ((4, 3), (4, 0), (5, 0)) with value 2
Finding Opponents best move: ((4, 3), (4, 0), (5, 0)) with value 5
Finding Opponents best move: ((5, 2), (4, 4), (4, 1)) with value 2
Finding Opponents best move: ((5, 2), (4, 4), (4, 1)) with value 5
Finding Opponents best move: ((4, 5), (4, 0), (5, 0)) with value 2
Finding Opponents best move: ((4, 5), (4, 0), (5, 2)) with value 1
Finding Opponents best move: ((4, 5), (4, 0), (5, 0)) with value 5
Finding Opponents best move: ((4, 5), (4, 0), (5, 2)) with value 4
Finding Opponents best move: ((4, 5), (4, 4), (4, 1)) with value 2
Finding Opponents best move: ((4, 5), (5, 3), (4, 1)) with value 1
Finding Opponents best move: ((4, 5), (4, 4), (4, 1)) with value 5
Finding Opponents best move: ((4, 5), (5, 3), (4, 1)) with value 4
Finding Opponents best move: ((4, 5), (5, 3), (4, 1)) with val

### Run Time

Minimax Search Depth == 1: Run Time ~ 59 Seconds
Minimax Search Depth == 2: Run Time ~1245  Seconds

Number of Moves Computed:
* Search Depth == 0: 46 Possible Moves (49 Possible moves from start - 3 by first CPU move)
* Search Depth == 1: 45! * 44! * 43!

In [51]:
from interactive_board_viz import PlayInteractiveGame
from isolation import Board
# from board_viz import InteractiveGame
from custom_player import CustomAIPlayer

##### NOTE: Need to click the squares order in which they move

# Remove the first argument ('RandomPlayer()') entirely to play against another HumanPlayer
ai_player = CustomAIPlayer(search_depth=1)
ig = PlayInteractiveGame(player1 = RandomPlayer(), opponent = ai_player, show_legal_moves=True)
# ig = InteractiveGame(opponent = RandomPlayer(), show_legal_moves=True)

GridspecLayout(children=(Button(description=' ', layout=Layout(grid_area='widget001', height='auto', width='au…

Output(layout=Layout(border='1px solid black'))

Button(description='Start', layout=Layout(height='50', width='100'), style=ButtonStyle())

Output()

In [50]:
ai_player.count

0

In [31]:
from IPython.display import display
button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("Button clicked.")

button.on_click(on_button_clicked)

NameError: name 'widgets' is not defined

# TODO

- Try iterative deepening approach
- Implement Alpha-Beta Pruning
- Try using transposition tables